In [1]:
%matplotlib qt

In [2]:
from main import *
import math

def constant_h(k: int):
    return 0.1

h0 = 1

def geometric_h(k: int):
    return h0 / 2**k

def exponential_decay(λ: float) -> LearningRateFunc:
    assert λ > 0
    return lambda k: h0 * math.exp(-λ * k)

def polynomial_decay(α: float, β: float) -> LearningRateFunc:
    assert α > 0
    assert β > 0
    return lambda k: 1/math.sqrt(k + 1) * (β * k + 1) ** -α


In [3]:
q1 = Quadratic(
    np.array([[1, 0], [0, 1]]),
    np.array([1, 1]),
    -1.4
)

q2 = Quadratic(
    np.array([[0.1, 0], [0, 3]]),
    np.array([0, 0]),
    0
)

In [ ]:
import numpy as np
from main import *

func = q1
x_0 = np.array([1.0, 1.0])
h = exponential_decay(0.5)

def relative_x_condition(x: np.ndarray, prev: np.ndarray) -> bool:
    # ‖𝑥_{𝑘+1} − 𝑥_𝑘‖ < 𝜀(‖𝑥_{𝑘+1}‖ + 1)
    eps = 1e-9
    return bool(np.linalg.norm(x - prev) < eps * (np.linalg.norm(x) + 1))

def relative_f_condition(func: BiFunc, x: np.ndarray, prev: np.ndarray) -> bool:
    # ‖∇𝑓(𝑥_𝑘)‖^2 < 𝜀‖∇𝑓(𝑥_0)‖^2
    eps = 1e-9
    return bool(np.linalg.norm(func.gradient(x) ** 2) < eps * np.linalg.norm(func.gradient(x_0)) ** 2)

trajectory = learning_rate_scheduling(x_0, func, h, relative_x_condition)

k: 42, x: [0.58047023 0.2213543 ], f: 0.18068774096269533


In [5]:
eps = 1e-9
trajectory = steepest_gradient_descent_dichotomy(x_0, func, eps, relative_x_condition)

k: 12, x: [ 7.24890207e-10 -8.05426782e-13], f: 5.2548527417457706e-20


In [6]:
eps = 1e-9
trajectory = steepest_gradient_descent_armijo(x_0, func, eps, relative_x_condition)

0.08601553829078802
0.08333458507836145
0.0833333347097293
0.08333333333486267
0.08333333333333504
0.08333333333333334
nan
nan
nan


/home/postmodernist1848/itmo/metopt/lab1/main.py:53: RuntimeWarning: overflow encountered in multiply
  x = x - h * grad
/home/postmodernist1848/itmo/metopt/lab1/main.py:36: RuntimeWarning: invalid value encountered in matmul
  return 2 * self.A @ x + self.B
/home/postmodernist1848/itmo/metopt/lab1/main.py:33: RuntimeWarning: invalid value encountered in matmul
  return float(x.T @ self.A @ x + self.B @ x + self.C)


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


KeyboardInterrupt: 

In [ ]:
from main import *
from scipy.optimize import minimize

minimize(
    func,
    x_0,
    method='CG',
    jac=func.gradient,
    options={'disp': True}
)

# Create a meshgrid for the 3D plot
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = func(np.array([X[i, j], Y[i, j]]))

# Plot the 3D surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.6) # type: ignore

# Plot the trajectory
ax.plot(trajectory[:, 0], trajectory[:, 1], [func(np.array([x, y])) for x, y in trajectory], color='r', marker='o')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z') # type: ignore
plt.show()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 6
         Gradient evaluations: 6


In [ ]:
from main import *
dichotomy(-10, 10, lambda x: (x - 2)**2 + 3.0, 1e-9)

1.9999999925494194